# Persistent Storage and Production Features with Agentune Simulate

  This notebook builds on the basic concepts from `getting_started.ipynb` to
  demonstrate production-ready features of Agentune Simulate.

  **Prerequisites**: Complete `getting_started.ipynb` first to understand the basics
  of conversation simulation.

  ## What you'll learn:
  - Set up persistent vector storage with Chroma for production use
  - Reuse vector stores across sessions to save time and resources
  - Handle larger datasets efficiently with persistent storage
  - Best practices for production deployments and scaling
  - Advanced configuration options for real-world scenarios

  ## Key Benefits of Persistent Storage:
  - **Performance**: No need to rebuild vector stores each session
  - **Scalability**: Handle thousands of conversations efficiently
  - **Cost Efficiency**: Reduce embedding computation costs through reuse
  - **Production Ready**: Suitable for deployment in production environments

## Installation

First, install the required dependencies:

In [1]:
!pip install -q langchain-chroma pandas
!pip install -q agentune-simulate

## Import Required Libraries

In [2]:
from pathlib import Path
import pandas as pd
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from agentune.simulate.models import Outcomes
from agentune.simulate.rag import conversations_to_langchain_documents
from agentune.simulate.simulation.session_builder import SimulationSessionBuilder

# Import example utilities
from utils import setup_logging_and_asyncio, load_conversations_from_csv, extract_outcomes_from_conversations

## Setup and API Key Configuration

This example uses OpenAI models, but any LangChain-compatible LLM can be supported. Configure your API key for the model provider you choose:

In [3]:
import os
import getpass

# Set up OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

print("✓ API key configured")

✓ API key configured


In [4]:
# Configure logging and asyncio for Jupyter
setup_logging_and_asyncio()

✓ Logging configured
✓ Asyncio event loop configured for Jupyter


## Load and Explore Sample Data

**Important**: For any data format or source, you must convert your data to `Conversation` objects for the simulator to work.

This example shows loading from CSV format:

In [5]:
# load_conversations_from_csv is an example utility function that converts CSV data to Conversation objects
# Example data is based on dhc2 dataset
# You need to implement a similar function for your data format and schema
conversations = load_conversations_from_csv("data/sample_conversations.csv")

print(f"Loaded {len(conversations)} conversations")
print(f"Sample conversation has {len(conversations[0].messages)} messages")
print(f"First message: {conversations[0].messages[0].content[:100]}...")

Loaded 100 conversations
Sample conversation has 6 messages
First message: Last night, I waited in line for 2 hours in the business office, but because I only had a copy of my...


In [6]:
# Explore the data structure
df = pd.read_csv("data/sample_conversations.csv")
print("Dataset overview:")
print(f"- Total messages: {len(df)}")
print(f"- Unique conversations: {df['conversation_id'].nunique()}")
print(f"- Message distribution: {df['sender'].value_counts().to_dict()}")
print(f"- Outcome distribution: {df['outcome_name'].value_counts().to_dict()}")

df.head()

Dataset overview:
- Total messages: 409
- Unique conversations: 100
- Message distribution: {'customer': 228, 'agent': 181}
- Outcome distribution: {'resolved': 365, 'unresolved': 44}


,conversation_id,sender,content,timestamp,outcome_name,outcome_description
0,conv_001,customer,"Last night, I waited in line for 2 hours in th...",2024-01-15T09:00:00+00:00,resolved,Issue was successfully resolved
1,conv_001,agent,"We’re very sorry, I am the Guangdong Customer ...",2024-01-15T09:02:00+00:00,resolved,Issue was successfully resolved
2,conv_001,customer,How can consumers supervise you if you don't s...,2024-01-15T09:05:00+00:00,resolved,Issue was successfully resolved
3,conv_001,agent,We will continue to improve various services a...,2024-01-15T09:09:00+00:00,resolved,Issue was successfully resolved
4,conv_001,customer,Nonsense. China Telecom has failed to make pro...,2024-01-15T09:14:00+00:00,resolved,Issue was successfully resolved


## Extract Outcomes for Simulation

Extract unique outcomes that our simulation will try to achieve:

In [7]:
# Extract unique outcomes from conversations
# Alternatively, you can define outcomes manually if you know them in advance
unique_outcomes = extract_outcomes_from_conversations(conversations)
outcomes = Outcomes(outcomes=tuple(unique_outcomes))

print(f"Found {len(unique_outcomes)} unique outcomes:")
for outcome in unique_outcomes:
    print(f"- {outcome.name}: {outcome.description}")

Found 2 unique outcomes:
- resolved: Issue was successfully resolved
- unresolved: Issue was not resolved


**Note**: You can also define outcomes manually if you know them in advance, instead of extracting them from existing conversations.

## Create Models and Vector Store

Chroma is a popular vector store for production use, allowing you to store vector data persistently and reuse it across sessions. Other LangChain-compatible vector stores can also be used.

For production use, you'll want persistent vector storage. Here's how to use Chroma:

In [8]:
# Setup models - OpenAI models work well, other LangChain-compatible models can also be used
# Note: gpt-4o has been tested and performs best for realistic conversations
chat_model = ChatOpenAI(model="gpt-4o", temperature=0.7)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Use all conversations for vector store training (simplified approach)
# Advanced: you could split data to reserve some conversations for validation
training_conversations = conversations
print(f"Using {len(training_conversations)} conversations for vector store training")

Using 100 conversations for vector store training


In [9]:
# Create persistent Chroma vector store
persist_directory = "./chroma_db"

chroma_store = Chroma(
    collection_name="conversation_examples",
    embedding_function=embeddings,
    persist_directory=persist_directory
)

documents = conversations_to_langchain_documents(conversations)
chroma_store.add_documents(documents)
print(f"✓ Added {len(documents)} documents to Chroma")

2025-07-23 17:03:39,346 - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


✓ Added 409 documents to Chroma


## Run Simulation

In [10]:
# Build session with Chroma vector store  
chroma_session = SimulationSessionBuilder(
    default_chat_model=chat_model,
    outcomes=outcomes,
    vector_store=chroma_store,
).build()

# Run simulation with Chroma
base_conversations = conversations[:5]
chroma_result = await chroma_session.run_simulation(real_conversations=base_conversations)

print("✓ Chroma simulation completed!")

2025-07-23 17:03:43,702 - Starting intent extraction on 5 conversations
2025-07-23 17:03:52,403 - Finished extracting original intents; generated 5 scenarios
2025-07-23 17:03:52,404 - Starting conversation simulations (self.max_concurrent_conversations=20)
2025-07-23 17:03:57,406 - Progress: 0/5 scenarios completed
2025-07-23 17:04:07,408 - Progress: 2/5 scenarios completed
2025-07-23 17:04:12,410 - Progress: 3/5 scenarios completed
2025-07-23 17:04:22,413 - Progress: 5/5 scenarios completed
2025-07-23 17:04:22,414 - Finished simulating conversations; simulated 5 conversations, with 0 failures
2025-07-23 17:04:22,415 - Starting analysis of simulation results
2025-07-23 17:04:31,594 - Finished analyzing results


✓ Chroma simulation completed!


In [11]:
# Compare results and save
print("=== CHROMA RESULTS ===")
print(chroma_result.generate_summary())

# Save results to file using built-in method
output_file = "chroma_simulation_results.json"
chroma_result.save_to_file(output_file)
print(f"\n✓ Results saved to {Path(output_file).resolve().relative_to(Path('../../').resolve())}")

=== CHROMA RESULTS ===
SIMULATION RESULTS
Session name: Simulation Session
Original conversations: 5
Simulated conversations: 5

Average messages per conversation:
  Original: 4.4
  Simulated: 2.8

Outcome distribution comparison:
Outcome              Original        Simulated      
--------------------------------------------------
resolved               4 (80.0%)     3 (60.0%)
unresolved             1 (20.0%)     2 (40.0%)

Sample conversation (4 messages):
  1. customer: Last night, I waited in line for 2 hours in the business office, but because I only had a copy of my...
  2. agent: I apologize for the inconvenience and frustration you've experienced with the cancellation process. ...
  3. customer: I understand the need for the original ID and the set-top box now, but my issue is with the lack of ...
  4. agent: I completely understand your frustration, and I apologize for the inconvenience this has caused. I w...

✓ Results saved to agentune_simulate/examples/chroma_simulation_r

## Next Steps

Now that you've completed a basic simulation:

1. **Use your own data**: Load your own conversations as a list of `Conversation` objects and use them to set up the simulation. See this example for loading conversations from tabular data: [Loading Conversations Example](https://github.com/SparkBeyond/agentune/blob/main/agentune_simulate/examples/loading_conversations.ipynb).
2. **Explore advanced features**: Check out the full documentation for more options caching of LLM responses, LLM failures handling, and more.

### Resources:
- [Full documentation](https://github.com/SparkBeyond/agentune/blob/main/agentune_simulate/README.md)
- [Advanced LLM config](https://github.com/SparkBeyond/agentune/blob/main/agentune_simulate/docs/langchain.md)
- [Complete examples](https://github.com/SparkBeyond/agentune/tree/main/agentune_simulate/examples)
- [Streamlit web interface](https://github.com/SparkBeyond/agentune/blob/main/agentune_simulate/streamlit/README.md)